In [193]:
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim

from sklearn.svm import SVR
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_absolute_error

In [194]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [195]:
file_path = '/content/drive/MyDrive/Colab Notebooks/ML/Lab 9/lateness_data.json'
data = pd.read_json(file_path)
data.head()

,direct_delivery,batched_pickup,transport_type,order_time,delivery_distance,order_preparation_time,responsible_id,store_latitude,store_longitude,client_latitude,client_longitude,status,status_time
0,yes,yes,automobile,2023-10-09 19:23:55,7798,10,4444,55.795518,37.631224,55.780525,37.700847,early,18
1,yes,yes,automobile,2023-07-31 11:43:13,553,10,3798,55.783786,37.624401,55.781943,37.628641,early,6
2,yes,yes,bicycle,2023-08-21 19:35:37,711,20,7595,55.729464,37.692976,55.732003,37.689528,early,9
3,yes,yes,automobile,2023-09-06 00:19:29,3538,10,3797,55.731702,37.581492,55.726069,37.604986,early,16
4,yes,yes,automobile,2023-09-06 19:23:28,4169,10,9509,55.781360,37.677339,55.787238,37.700311,early,5


## Self-practice

Using 10% sample from 2nd assignment dataset (task 1) and `status_time` as target variable (**note** for `early delivery` set `status_time` to negative value). Apply the following models:

1. Regression tree
2. SVM (use kernel of your choice)
3. Simple Neural Network (with two or three hidden layers)

Compare the ML models using the appropriate metrics

In [196]:
data_sample = data.sample(frac=0.1, random_state=42)

In [197]:
status_mapping = {
    'early': 2,
    'late': 0,
    'on time': 1
}

data_sample['status'] = data_sample['status'].map(status_mapping)

In [198]:
data_sample = pd.get_dummies(data_sample, columns=['direct_delivery'], drop_first=True)
data_sample = pd.get_dummies(data_sample, columns=['batched_pickup'], drop_first=True)

In [199]:
transport_type_mapping = {
    'foot': 0,
    'scooter': 1,
    'bicycle': 2,
    'automobile': 3
}
data_sample['transport_type'] = data_sample['transport_type'].map(transport_type_mapping)

In [200]:
data_sample.sort_values(by='order_time', inplace=True)
data_sample.drop('order_time', axis=1, inplace=True)

In [201]:
data_sample.loc[data_sample['status'] == 2, 'status_time'] = -data_sample['status_time']

In [202]:
X = data_sample.drop(['status_time'], axis=1)
y = data_sample[['status_time']]

In [203]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

# Regression Tree:

In [204]:
reg_tree_model = DecisionTreeRegressor(random_state=42)

In [205]:
reg_tree_model.fit(X_train, y_train)

DecisionTreeRegressor(random_state=42)

In [206]:
predictions_reg_tree = reg_tree_model.predict(X_test)

In [207]:
mae_reg_tree = mean_absolute_error(y_test, predictions_reg_tree)
print(f"Mean Absolute Error (Regression Tree): {mae_reg_tree}")

Mean Absolute Error (Regression Tree): 4.993682310469314


# SVM (Support Vector Machine):

In [208]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [209]:
svm_model = SVR(kernel='rbf')

In [210]:
svm_model.fit(X_train_scaled, y_train)

/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


SVR()

In [211]:
predictions_svm = svm_model.predict(X_test_scaled)

In [212]:
mae_svm = mean_absolute_error(y_test, predictions_svm)
print(f"Mean Absolute Error (SVM): {mae_svm}")

Mean Absolute Error (SVM): 3.7713590628066074


# Simple Neural Network (with two or three hidden layers)

In [213]:
class SNN(nn.Module):
    def __init__(self, input_dim):
        super(SNN, self).__init__()

        # Define hidden layers
        self.hidden_layer1 = nn.Linear(input_dim, 128)
        self.hidden_layer2 = nn.Linear(128, 64)
        self.hidden_layer3 = nn.Linear(64, 32)
        self.hidden_layer4 = nn.Linear(32, 16)

        # Output layers for classification and regression
        self.regression_output = nn.Linear(16, 1)

    def forward(self, x):
        # Apply ReLU activation to hidden layers
        x = torch.relu(self.hidden_layer1(x))
        x = torch.relu(self.hidden_layer2(x))
        x = torch.relu(self.hidden_layer3(x))
        x = torch.relu(self.hidden_layer4(x))

        return self.regression_output(x)

In [214]:
X_train_tensor = torch.tensor(X_train.values, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train.values, dtype=torch.float32)

In [215]:
input_dim = X_train.shape[1]  # Adjust this based on your input data
model = SNN(input_dim)

In [216]:
criterion = nn.MSELoss()  # Mean Squared Error for regression
optimizer = optim.Adam(model.parameters(), lr=0.001)  # Adjust the learning rate

In [217]:
num_epochs = 50

In [218]:
model.train()

SNN(
  (hidden_layer1): Linear(in_features=11, out_features=128, bias=True)
  (hidden_layer2): Linear(in_features=128, out_features=64, bias=True)
  (hidden_layer3): Linear(in_features=64, out_features=32, bias=True)
  (hidden_layer4): Linear(in_features=32, out_features=16, bias=True)
  (regression_output): Linear(in_features=16, out_features=1, bias=True)
)

In [219]:
for epoch in range(num_epochs):
    # Forward pass
    outputs = model(X_train_tensor)

    # Calculate loss
    loss = criterion(outputs, y_train_tensor.view(-1, 1))

    # Backward pass and optimization
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    # Print training information
    if (epoch + 1) % 10 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

Epoch [10/50], Loss: 159.7687
Epoch [20/50], Loss: 140.7382
Epoch [30/50], Loss: 125.2242
Epoch [40/50], Loss: 123.9836
Epoch [50/50], Loss: 123.8141


In [220]:
model.eval()

SNN(
  (hidden_layer1): Linear(in_features=11, out_features=128, bias=True)
  (hidden_layer2): Linear(in_features=128, out_features=64, bias=True)
  (hidden_layer3): Linear(in_features=64, out_features=32, bias=True)
  (hidden_layer4): Linear(in_features=32, out_features=16, bias=True)
  (regression_output): Linear(in_features=16, out_features=1, bias=True)
)

In [221]:
X_test_tensor = torch.tensor(X_test.values, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test.values, dtype=torch.float32)

In [222]:
with torch.no_grad():
    predictions = model(X_test_tensor)

In [223]:
mae_snn = mean_absolute_error(predictions, y_test_tensor.view(-1, 1))
print(f"Mean Absolute Error (SNN): {mae_snn}")

Mean Absolute Error (SNN): 9.227137565612793


Lower MAE values indicate better model performance.

Here's how you can interpret the results:

- **Regression Tree (Decision Tree):**
  - MAE: 4.99
  - Interpretation: On average, the predictions from the Regression Tree are approximately 4.99 units away from the actual values.

- **Support Vector Machine (SVM):**
  - MAE: 3.77
  - Interpretation: On average, the predictions from the SVM are approximately 3.77 units away from the actual values.

- **Simple Neural Network (SNN):**
  - MAE: 9.23
  - Interpretation: On average, the predictions from the Simple Neural Network are approximately 9.23 units away from the actual values.

**Comparison:**
- The SVM has the lowest MAE among the three models, suggesting that, based on MAE alone, the SVM is performing better in terms of absolute prediction accuracy.
- The Regression Tree has a higher MAE than the SVM but lower than the Simple Neural Network.
- The Simple Neural Network has the highest MAE among the three models, indicating that, in this evaluation, it has the least accurate predictions on average.